In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import json_lines as jl

In [4]:
limit = 50
train_file = "../data/raw/train_dataset.jl.gz"

with jl.open(train_file) as file:
    train_json = [next(file) for _ in range(0,50)]

In [7]:
train_json[1]

{'item_bought': 228737,
 'user_history': [{'event_info': 643652,
   'event_timestamp': '2019-10-06T18:02:53.893-0400',
   'event_type': 'view'},
  {'event_info': 'DESMAMADEIRA ELETRICA',
   'event_timestamp': '2019-10-07T09:45:29.322-0400',
   'event_type': 'search'},
  {'event_info': 1156086,
   'event_timestamp': '2019-10-07T09:46:07.960-0400',
   'event_type': 'view'},
  {'event_info': 'DESMAMADEIRA ELETRICA',
   'event_timestamp': '2019-10-07T09:46:17.100-0400',
   'event_type': 'search'},
  {'event_info': 'DESMAMADEIRA ELETRICA',
   'event_timestamp': '2019-10-07T09:46:19.173-0400',
   'event_type': 'search'},
  {'event_info': 1943604,
   'event_timestamp': '2019-10-07T09:47:53.958-0400',
   'event_type': 'view'},
  {'event_info': 'DESMAMADEIRA ELETRICA',
   'event_timestamp': '2019-10-07T18:53:20.113-0400',
   'event_type': 'search'},
  {'event_info': 206667,
   'event_timestamp': '2019-10-07T18:53:26.670-0400',
   'event_type': 'view'},
  {'event_info': 'DESMAMADEIRA ELETRICA',
